In [4]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
# bin50 = sc.read_h5ad('/home/hanyuji/Data/ST_data/macaque_hippocampus/bin50/macaque1/T25.h5ad')
# cell = sc.read_h5ad('/home/hanyuji/Data/ST_data/macaque_hippocampus/cell/macaque1/T25.h5ad')
# clus = sc.read_h5ad('/home/hanyuji/Data/ST_data/macaque_hippocampus/clus/macaque1/T27.h5ad')

In [12]:
def random_subset(adata,cell_num=1200):
    random_indices = np.random.choice(adata.n_obs, cell_num, replace=True)   # 从 276593 个细胞中随机选取 3000 个细胞的索引
    adata_subset = adata[random_indices, :]   # 使用选取的索引创建新的 AnnData 对象

    return adata_subset

In [20]:
dataset_dir = '/home/hanyuji/Data/ST_data/macaque_hippocampus/cell/macaque1/'

cell_macaque1 = ['T25.h5ad', 'T29.h5ad', 'T32.h5ad', 'T36.h5ad', 'T40.h5ad', 'T43.h5ad', 'T47.h5ad', 'T50.h5ad', 'T53.h5ad', 'T56.h5ad', 'T27.h5ad',
 'T30.h5ad', 'T33.h5ad', 'T37.h5ad', 'T41.h5ad', 'T44.h5ad', 'T48.h5ad', 'T51.h5ad', 'T54.h5ad', 'T57.h5ad', 'T28.h5ad', 'T31.h5ad',
 'T34.h5ad', 'T39.h5ad', 'T42.h5ad', 'T45.h5ad', 'T49.h5ad', 'T52.h5ad', 'T55.h5ad', 'T59.h5ad']

adatas = []
for file_name in tqdm(cell_macaque1):
    adata = sc.read_h5ad(dataset_dir+file_name)
    sc.pp.filter_genes(adata, min_counts = 5)
    subset = random_subset(adata,1000)
    adatas.append(subset)

combined_adata = ad.concat(
    adatas, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    label='batch', 
    keys=list(range(1, len(adatas)+1)), 
    index_unique='-', 
    fill_value=0
)


100%|██████████| 30/30 [00:06<00:00,  4.47it/s]
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [19]:
combined_adata

AnnData object with n_obs × n_vars = 90000 × 15275
    obs: 'region', 'cellID', 'x', 'y', 'cellType', 'batch'
    obsm: 'spatial'

In [21]:
sc.pp.highly_variable_genes(combined_adata, n_top_genes=2000 ,flavor='seurat_v3')
combined_adata = combined_adata[:, combined_adata.var.highly_variable]
# 标准化 log 
sc.pp.normalize_total(combined_adata, target_sum=1e4)
sc.pp.log1p(combined_adata)

/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:233: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


In [24]:
combined_adata.write('/home/hanyuji/Workbench/ST/paste_alignment_cortex/PASTE_align/macaque_hippocampus_cell_macaque1_30slices_1000spot_2000gene.h5ad')